# Using maketables with linearmodels

This notebook demonstrates how to create publication-ready regression tables from linearmodels panel data models using maketables.

## Setup

First, install the required packages if you haven't already:

```bash
pip install maketables linearmodels
```

In [1]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS, RandomEffects, PooledOLS
import maketables as mt


## Load Panel Data

We'll use wage panel data from linearmodels examples.

In [2]:
# Load wage panel data
from linearmodels.datasets import wage_panel

data = wage_panel.load()

# Set panel structure
data = data.set_index(['nr', 'year'])


## Example 1: Basic Panel Regression Comparison

Compare Pooled OLS, Fixed Effects, and Random Effects models.

In [3]:
# Pooled OLS (no fixed effects)
pooled = PooledOLS.from_formula("lwage ~ expersq + union + married", data)
pooled_res = pooled.fit(cov_type='clustered', cluster_entity=True)

# Fixed Effects (Entity)
fe = PanelOLS.from_formula("lwage ~ expersq + union + married  + EntityEffects", data)
fe_res = fe.fit(cov_type='clustered', cluster_entity=True)

# Random Effects
re = RandomEffects.from_formula("lwage ~ expersq + union + married  + EntityEffects", data)
re_res = re.fit(cov_type='clustered', cluster_entity=True)


In [4]:
# Create comparison table
table1 = mt.ETable(
    [pooled_res, fe_res, re_res],
    model_heads=['Pooled OLS', 'Fixed Effects', 'Random Effects'],
    caption='Wage Regression: Comparison of Panel Estimators',
    model_stats=['N', 'r2', 'r2_within'],
    notes='Clustered standard errors by entity in parentheses.'
)

table1

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab63e3c440>

## Example 2: Time and Entity Fixed Effects

Estimate models with different fixed effects specifications.

In [5]:
# Model 1: No fixed effects
mod1 = PanelOLS.from_formula('lwage ~ expersq + union + married', data)
res1 = mod1.fit(cov_type='clustered', cluster_entity=True)

# Model 2: Entity fixed effects
mod2 = PanelOLS.from_formula('lwage ~ expersq + union + married + EntityEffects', data)
res2 = mod2.fit(cov_type='clustered', cluster_entity=True)

# Model 3: Time fixed effects
mod3 = PanelOLS.from_formula('lwage ~ expersq + union + married + TimeEffects', data)
res3 = mod3.fit(cov_type='clustered', cluster_entity=True)

# Model 4: Both entity and time fixed effects
mod4 = PanelOLS.from_formula('lwage ~ expersq + union + married + EntityEffects + TimeEffects', data)
res4 = mod4.fit(cov_type='clustered', cluster_entity=True)

# Create table with variable labels
labels = {
    'expersq': 'Experience²',
    'union': 'Union Member',
    'married': 'Married',
    'lwage': 'Log(Wage)',
    'nr': 'Individual ID',
    'year': 'Year',
}

mt.ETable(
    [res1, res2, res3, res4],
    model_heads=['Pooled', 'Entity FE', 'Time FE', 'Two-way FE'],
    caption='Fixed Effects Specifications',
    labels=labels,
    model_stats=['N', 'r2', 'r2_within', 'r2_between'],
    show_fe=True,
    notes='Clustered standard errors by entity in parentheses.'
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab664b7b60>

## Example 3: Different Standard Error Specifications

In [6]:
# Same model, different standard errors
formula = 'lwage ~ expersq + union + married + EntityEffects'
mod = PanelOLS.from_formula(formula, data)

# Homoskedastic
res_hom = mod.fit(cov_type='unadjusted')

# Heteroskedastic-robust
res_robust = mod.fit(cov_type='robust')

# Clustered by entity
res_cluster = mod.fit(cov_type='clustered', cluster_entity=True)

print("Models with different SEs estimated!")

Models with different SEs estimated!


In [7]:
table3 = mt.ETable(
    [res_hom, res_robust, res_cluster],
    model_heads=['Homoskedastic', 'Robust', 'Clustered'],
    caption='Comparison of Standard Error Specifications',
    labels=labels,
    model_stats=['N', 'r2_within', 'se_type'],
    notes='Entity fixed effects included in all specifications.'
)

table3

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab664b7cb0>

In [8]:
## Example 4: High-Dimensional Fixed Effects with AbsorbingLS

from linearmodels import AbsorbingLS

# Create sample data with high-dimensional fixed effects
np.random.seed(42)

# Simulate firm-worker matched data
n_firms = 500
n_workers = 1000
n_periods = 8

# Generate matched panel
obs_per_match = 3
n_obs = n_firms * obs_per_match

firm_ids = np.repeat(np.arange(n_firms), obs_per_match)
worker_ids = np.random.choice(n_workers, size=n_obs)
year_ids = np.random.choice(n_periods, size=n_obs) + 2010

# Create data with firm and worker effects
firm_effects = np.random.randn(n_firms)
worker_effects = np.random.randn(n_workers)

df_hdfe = pd.DataFrame({
    'firm_id': firm_ids,
    'worker_id': worker_ids,
    'year': year_ids,
    'lwage': (firm_effects[firm_ids] + 
              worker_effects[worker_ids] + 
              np.random.randn(n_obs) * 0.5),
    'experience': np.random.randint(0, 30, n_obs),
    'training': np.random.binomial(1, 0.3, n_obs),
    'female': np.random.binomial(1, 0.45, n_obs)
})

print(f"Dataset: {len(df_hdfe)} observations")
print(f"Firms: {df_hdfe['firm_id'].nunique()}")
print(f"Workers: {df_hdfe['worker_id'].nunique()}")
print(f"Years: {df_hdfe['year'].nunique()}")

# Prepare data for AbsorbingLS
dependent = df_hdfe[['lwage']]
exog = df_hdfe[['experience', 'training', 'female']]

# Model 1: No fixed effects (baseline)
mod_ols = AbsorbingLS(dependent=dependent, exog=exog, absorb=None)
res_ols = mod_ols.fit(cov_type='robust')

# Model 2: Absorb firm fixed effects
mod_firm = AbsorbingLS(dependent=dependent, exog=exog, absorb=df_hdfe[['firm_id']])
res_firm = mod_firm.fit(cov_type='clustered', clusters=df_hdfe[['firm_id']])

# Model 3: Absorb both firm and year fixed effects
mod_firm_year = AbsorbingLS(dependent=dependent, exog=exog, absorb=df_hdfe[['firm_id', 'year']])
res_firm_year = mod_firm_year.fit(cov_type='clustered', clusters=df_hdfe[['firm_id']])

# Model 4: Absorb firm, year, and firm×year interaction
df_hdfe['firm_year'] = df_hdfe['firm_id'].astype(str) + '_' + df_hdfe['year'].astype(str)
mod_interaction = AbsorbingLS(dependent=dependent, exog=exog, absorb=df_hdfe[['firm_id', 'year', 'firm_year']])
res_interaction = mod_interaction.fit(cov_type='clustered', clusters=df_hdfe[['firm_id']])

print(f"\nOLS: {len(res_ols.params)} coefficients")
print(f"Firm FE absorbed: {len(res_firm.params)} coefficients shown ({df_hdfe['firm_id'].nunique()} absorbed)")
print(f"Firm+Year FE absorbed: {len(res_firm_year.params)} coefficients shown")
print(f"Firm×Year FE absorbed: {len(res_interaction.params)} coefficients shown")

# Create comparison table
hdfe_labels = {
    'experience': 'Experience (years)',
    'training': 'Training Program',
    'female': 'Female',
    'lwage': 'Log(Wage)'
}

mt.ETable(
    [res_ols, res_firm, res_firm_year, res_interaction],
    model_heads=['OLS', 'Firm FE', 'Firm+Year FE', 'Firm×Year FE'],
    caption='High-Dimensional Fixed Effects with AbsorbingLS',
    labels=hdfe_labels,
    model_stats=['N', 'r2', 'adj_r2'],
    notes='All models include experience, training, and gender. Models 2-4 progressively absorb high-dimensional fixed effects. Clustered standard errors by firm in models 2-4.'
)

Dataset: 1500 observations
Firms: 500
Workers: 755
Years: 8

OLS: 3 coefficients
Firm FE absorbed: 3 coefficients shown (500 absorbed)
Firm+Year FE absorbed: 3 coefficients shown
Firm×Year FE absorbed: 3 coefficients shown


<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab665ac050>

In [9]:
## Example 5: Instrumental Variables Regression

from linearmodels.iv import IV2SLS
from linearmodels.datasets import meps

# Load MEPS data (cross-sectional, not panel)
meps_data = meps.load()
print(f"MEPS dataset: {len(meps_data)} observations")

# For IV2SLS, we can use formulas directly without PooledOLS
# The data is cross-sectional, not panel data

# 2SLS with single instrument (SSI ratio)
iv1 = IV2SLS.from_formula('ldrugexp ~ 1 + totchr + age + female + blhisp + linc + [hi_empunion ~ ssiratio]', data=meps_data)
iv1_res = iv1.fit(cov_type='robust')

# 2SLS with multiple instruments (SSI ratio, multlc, firmsz)
iv2 = IV2SLS.from_formula('ldrugexp ~ 1 + totchr + age + female + blhisp + linc + [hi_empunion ~ ssiratio + multlc + firmsz]', data=meps_data)
iv2_res = iv2.fit(cov_type='robust')

# For comparison, estimate "OLS" using IV2SLS without instruments (same as OLS)
ols_iv = IV2SLS.from_formula('ldrugexp ~ 1 + hi_empunion + totchr + age + female + blhisp + linc', data=meps_data)
ols_res = ols_iv.fit(cov_type='robust')


# Create comparison table
iv_labels = {
    'hi_empunion': 'Health Insurance (Employer/Union)',
    'linc': 'Log(Income)',
    'age': 'Age',
    'female': 'Female',
    'blhisp': 'Black/Hispanic',
    'totchr': 'Chronic Conditions',
    'ldrugexp': 'Log(Drug Expenditure)'
}

mt.ETable(
    [ols_res, iv1_res, iv2_res],
    model_heads=['OLS', 'IV (SSI)', 'IV (SSI+multlc+firmsz)'],
    caption='Instrumental Variables: Health Insurance and Drug Expenditure',
    labels=iv_labels,
    model_stats=['N', 'r2'],
)

MEPS dataset: 10391 observations


c:\PythonProjects\maketables\.pixi\envs\linearmodels\Lib\site-packages\linearmodels\iv\model.py:560: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
c:\PythonProjects\maketables\.pixi\envs\linearmodels\Lib\site-packages\linearmodels\iv\model.py:560: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
c:\PythonProjects\maketables\.pixi\envs\linearmodels\Lib\site-packages\linearmodels\iv\model.py:560: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab665acd70>

In [10]:

# Use models from Example 2 (Fixed Effects specifications)
# Display 95% confidence intervals instead of standard errors
mt.ETable(
    [res1, res2, res3, res4],
    model_heads=['Pooled', 'Entity FE', 'Time FE', 'Two-way FE'],
    caption='Fixed Effects with 95% Confidence Intervals',
    labels=labels,
    coef_fmt="b \n [ci95l, ci95u]",
    digits=3,
    model_stats=['N', 'r2_within'],
    notes='95% confidence intervals in brackets. Clustered standard errors by entity.'
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1ab63e3c590>